# Compute the pairwise distance between bacteria based on their defense arsenal

From the defense systems genes table (the one specifying each gene in each defense system of each strain), computes the pairwise distance (jaccard) between the defense arsenal of each couple of strains. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

plt.style.use("ggplot")
np.random.seed(0)

In [2]:
bact_defense_genes = pd.read_csv("defense_finder_genes_with_clusters.csv", sep=";").sort_values(["bacteria", "sys_id"]).set_index("bacteria")
bact_defense_genes.head(5)

,sys_id,type,subtype,hit_id,gene_name,cluster_80_80,cluster_99_99,cluster_999_999
bacteria,,,,,,,,
001-023,ESCO.0423.00107.0005i_MazEF_2274,MazEF,MazEF,ESCO.0423.00107.0005i_01434,MazEF__MazE,ESCO.0423.00001.0001i_05294,ESCO.0423.00003.0001i_04717,ESCO.0423.00003.0001i_04717
001-023,ESCO.0423.00107.0005i_MazEF_2274,MazEF,MazEF,ESCO.0423.00107.0005i_01435,MazEF__MazF,ESCO.0423.00001.0001i_05295,ESCO.0423.00001.0001i_05295,ESCO.0423.00003.0001i_04718
001-023,ESCO.0423.00107.0005i_RM_Type_IV_3206,RM,RM_Type_IV,ESCO.0423.00107.0005i_01563,RM_Type_IV__Type_IV_REases,ESCO.0423.00003.0001i_04834,ESCO.0423.00107.0005i_01563,ESCO.0423.00107.0005i_01563
001-023,ESCO.0423.00107.0005i_RM_Type_I_3205,RM,RM_Type_I,ESCO.0423.00107.0005i_01566,RM__Type_I_MTases,ESCO.0423.00001.0001i_05396,ESCO.0423.00001.0001i_05396,ESCO.0423.00107.0005i_01566
001-023,ESCO.0423.00107.0005i_RM_Type_I_3205,RM,RM_Type_I,ESCO.0423.00107.0005i_01567,RM__Type_I_REases,ESCO.0423.00001.0001i_05397,ESCO.0423.00001.0001i_05397,ESCO.0423.00107.0005i_01567


In [3]:
class DefenseSystem:
    """
    Class for encoding defense systems with potentially multple genes. This class allows to assess
    the equality between defense systems in two distinct bacteria, by comparing all the genes composing
    the defense system, and assessing the equality of each particular element. DefenseSystems objects
    are also hashable, which makes it possible to store them in sets and efficiently compute set 
    operations on defense arsenals across bacteria (sets of DefenseSystems).
    """

    def __init__(self, df_sys):
            
        if type(df_sys) == pd.DataFrame:
            system, subsystem, sys_id = df_sys["type"].unique()[0], df_sys["subtype"].unique()[0], df_sys["sys_id"].unique()[0]
            cluster_80_80, cluster_99_99 = df_sys["cluster_80_80"], df_sys["cluster_99_99"]
            self.cluster_80_80 = set(cluster_80_80.values)
            self.cluster_99_99 = set(cluster_99_99.values)
        else:  # only 1 system, with a single gene
            system, subsystem, sys_id = df_sys["type"], df_sys["subtype"], df_sys["sys_id"]
            cluster_80_80, cluster_99_99 = df_sys["cluster_80_80"], df_sys["cluster_99_99"]
            self.cluster_80_80 = set(cluster_80_80)
            self.cluster_99_99 = set(cluster_99_99)
            
        self.sys_id = sys_id
        self.system = system
        self.subsystem = subsystem
        
        if type(self.cluster_80_80) == list and type(self.cluster_99_99) == list:
            assert len(self.cluster_80_80) == len(self.cluster_99_99), print(len(self.cluster_80_80), len(self.cluster_99_99))

        self.size = len(self.cluster_80_80)

    def __eq__(self, other):
        return self.cluster_80_80 == other.cluster_80_80
    
    def __str__(self):
        return self.subsystem + "_" + str(self.__hash__()).strip("-")[:4]
    
    def __hash__(self):
        s = "".join([gene for gene in self.cluster_80_80])
        return hash(s)
    
    def __repr__(self):
        if self.size == 1:
            return self.subsystem + "_" + str(self.__hash__()).strip("-")[:4]# + ' (' + str(self.size) + " gene)"
        else:
            return self.subsystem + "_" + str(self.__hash__()).strip("-")[:4]# + ' (' + str(self.size) + " genes)"

In [4]:
# test the DefenseSystem class
b1 = bact_defense_genes.loc["NILS25"]
b2 = bact_defense_genes.loc["NILS27"]

b1_arsenal = {DefenseSystem(b1.loc[b1["sys_id"] == sys]) for sys in b1["sys_id"].unique()}
b2_arsenal = {DefenseSystem(b2.loc[b2["sys_id"] == sys]) for sys in b2["sys_id"].unique()}

# identify shared systems
shared = {}
for ds in b2_arsenal:
    if ds in b1_arsenal:
        shared[ds] = True
    else:
        shared[ds] = False
    
# shared
print(b1_arsenal.intersection(b2_arsenal))
print(b2_arsenal.union(b1_arsenal))
print(b2_arsenal.symmetric_difference(b1_arsenal))

{SanaTA_5074, PsyrTA_4316, RM_Type_II_7491, RM_Type_I_4300, MazEF_7953, Mok_Hok_Sok_3857, RM_Type_IV_2402, Retron_I_C_4030}
{MazEF_8759, PsyrTA_4316, RM_Type_II_7491, Avs_IV_6257, Lamassu-Mrr_7056, SanaTA_5074, RM_Type_I_5981, RM_Type_I_4300, MazEF_7953, RM_Type_I_8975, Mok_Hok_Sok_3857, AbiE_7774, RM_Type_IV_2402, Retron_I_C_4030, PD-T4-3_5054}
{MazEF_8759, Avs_IV_6257, RM_Type_I_5981, RM_Type_I_8975, Lamassu-Mrr_7056, AbiE_7774, PD-T4-3_5054}


In [5]:
# Build the defense arsenal of each bacteria
all_defense_sys = []
all_arsenals = {}
for i, bact in enumerate(sorted(bact_defense_genes.index.unique())):
    all_defense_genes_in_bact = bact_defense_genes.loc[bact]

    if type(all_defense_genes_in_bact["sys_id"]) == pd.Series:  # multiple systems -> list of sys_id
        all_arsenals[bact] = {DefenseSystem(all_defense_genes_in_bact.loc[all_defense_genes_in_bact["sys_id"] == sys]) for sys in all_defense_genes_in_bact["sys_id"].unique()}
    else:  # unique system with a single gene
        all_arsenals[bact] = {DefenseSystem(all_defense_genes_in_bact)}

    for d in all_arsenals[bact]:
        if not d in all_defense_sys:
            all_defense_sys.append(d)

print(len(all_arsenals))

do_save = False
if do_save:
    pickle.dump(all_arsenals, open("defense_arsenal_370+host.pickle", "wb"))

404


In [6]:
# Compute pairwise distance between bacterial arsenals
pairwise_jac = []
for i, b1 in enumerate(sorted(bact_defense_genes.index.unique())):
    for j, b2 in enumerate(sorted(bact_defense_genes.index.unique()[:])):
        arsenal_1, arsenal_2 = all_arsenals[b1], all_arsenals[b2]

        intersection = len(arsenal_1.intersection(arsenal_2))
        union = len(arsenal_1.union(arsenal_2))
        jaccard = (1 - intersection / union)

        pairwise_jac.append({"b1": b1, "b2": b2, 
                             "arsenal_size1": len(arsenal_1), "arsenal_size2": len(arsenal_2), 
                             "intersection": intersection, "union": union, "jaccard_distance": jaccard})

pairwise_jac = pd.DataFrame(pairwise_jac).set_index(["b1", "b2"])

do_save_df = False
if do_save_df:
    pairwise_jac.to_csv("coli_ds_distances_cluster_80_80_dist=jaccard.csv", sep=";", index=True)

### Compute the defense systems table

In [7]:
all_sys_df = []
for d in all_defense_sys:
    all_bact_with_system = []
    for b in all_arsenals:
        if d in all_arsenals[b]:
            all_sys_df.append({"system": str(d), "bacteria": b, "presence": 1})
all_sys_df = pd.DataFrame(all_sys_df).pivot(index="bacteria",  columns="system", values="presence").fillna(0)

do_save = False
if do_save:
    all_sys_df.to_csv("370+host_defense_systems_cluster_80_80.csv", sep=";")
all_sys_df

system,AbiD_1877,AbiD_2873,AbiD_3671,AbiD_4519,AbiE_3231,AbiE_7774,AbiE_9209,AbiH_5665,AbiJ_2287,AbiL_6153,...,dCTPdeaminase_6314,dCTPdeaminase_7584,dCTPdeaminase_8162,dCTPdeaminase_8252,dCTPdeaminase_8889,dCTPdeaminase_8959,dGTPase_5869,dGTPase_6242,radar_II_2402,radar_I_1045
bacteria,,,,,,,,,,,,,,,,,,,,,
001-023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001-031-c1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
003-026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
013-008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
025-010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TW10509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Compute the defense systems family table

In [8]:
all_sys_fam = []
for d in all_defense_sys:
    all_bact_with_system = []
    for b in all_arsenals:
        if d in all_arsenals[b]:
            all_sys_fam.append({"system": d.subsystem, "bacteria": b, "presence": 1})
all_sys_fam = pd.DataFrame(all_sys_fam).groupby(["bacteria", "system"]).sum().reset_index()

all_sys_fam = all_sys_fam.pivot(index="bacteria",  columns="system", values="presence").fillna(0)

do_save = False
if do_save:
    all_sys_fam.astype(int).to_csv("370+host_defense_systems_subtypes.csv", sep=";")

all_sys_fam

system,AbiD,AbiE,AbiH,AbiJ,AbiL,AbiP2,AbiQ,AbiU,Avs_II,Avs_IV,...,SspBCDE,Thoeris_I,Thoeris_II,Wadjet_III,Zorya_TypeI,Zorya_TypeII,dCTPdeaminase,dGTPase,radar_I,radar_II
bacteria,,,,,,,,,,,,,,,,,,,,,
001-023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001-031-c1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
003-026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
013-008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
025-010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TW10509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Visualize defense systems table

In [9]:
bact_features = pd.read_csv("../picard_collection.csv", sep=";").set_index("bacteria")
bacteria_order = np.loadtxt("../panacota/tree/370+host_leaf_order_root=albertii.txt", delimiter=",", dtype=str)[::-1]  # load bacteria order

all_sys_fam = pd.read_csv("370+host_defense_systems_subtypes.csv", sep=";").set_index("bacteria")

In [10]:
all_sys_fam

,AbiD,AbiE,AbiH,AbiJ,AbiL,AbiP2,AbiQ,AbiU,Avs_II,Avs_IV,...,SspBCDE,Thoeris_I,Thoeris_II,Wadjet_III,Zorya_TypeI,Zorya_TypeII,dCTPdeaminase,dGTPase,radar_I,radar_II
bacteria,,,,,,,,,,,,,,,,,,,,,
001-023,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
001-031-c1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
003-026,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
013-008,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
025-010,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T147,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
T205,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TW10509,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
from bokeh.models import BasicTicker, PrintfTickFormatter, HoverTool, TextInput, AutocompleteInput, ColumnDataSource, CustomJS
from bokeh.plotting import figure, show
from bokeh.sampledata.unemployment1948 import data
from bokeh.transform import linear_cmap
from bokeh.io import export_svg, export_png

bacteria = list(bacteria_order)
ds_list = list(all_sys_fam.columns)

df = all_sys_fam.copy()
df = pd.DataFrame(df.stack(), columns=["score"]).reset_index().rename({"bacteria": "Bacteria", "level_1": "Defense_System"}, axis=1)

# add bacterial and phage genomic features
df = pd.merge(left=df, right=bact_features[["Clermont_Phylo", "ST_Warwick", "LPS_type", "O-type", "H-type", "Klebs_capsule_type"]], left_on="Bacteria", right_index=True)
df = df.rename({"O-type": "O", "H-type": "H", "Klebs_capsule_type": "K"}, axis=1)

# Choose colormap that suits the interaction matrix type
# colors = ["#B8E0ED", "#8ABBD6", "#4F7BB6", "#214D96", "#0C2471", "#060E34"]  # lively blue
colors = ["#ffffff", "#0C2471"]

hover = """
    <div>
        <div>
            <span style="font-size: 15px; font-weight: bold;">@Bacteria vs. @Defense_System</span>
        </div>
        <div>
            <span style="font-size: 14px; color: #119; font-weight: bold;">Score: @score</span>
        </div>
        <div>
            <span style="font-size: 10px; color: #911;">Phylogroup: @Clermont_Phylo \n</span>
        </div>
        <div>
            <span style="font-size: 10px; color: #911;">ST: @ST_Warwick \n</span>
        </div>
        <div>
           <span style="font-size: 10px; color: #911;">LPS: @LPS_type \n</span>
        </div>
        <div>
           <span style="font-size: 10px; color: #911;">O-type: @O \n</span>
        </div>
        <div>
             <span style="font-size: 10px; color: #911;">H-type: @H \n</span>
        </div>
        <div>
             <span style="font-size: 10px; color: #911;">K-type: @K \n</span>
        </div>
"""

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

p = figure(title=f"",
           x_range=bacteria, y_range=ds_list,
           x_axis_location="above", width=2500, height=1250, tools=TOOLS,
           tooltips=hover)

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "10px"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = np.pi / 3
p.axis.visible = True

r = p.rect(x="Bacteria", y="Defense_System", width=1, height=1, source=df,
           fill_color=linear_cmap("score", colors, low=0, high=1),
           line_color="#bbbbbb", line_width=0.10)

show(p)